In [1]:
# Import library
import pandas as pd
from sqlalchemy import create_engine

# EXTRACT & TRANSFORM

Bees Colony Census Data By County

In [2]:
# Store beeColonyCensusDataByCounty CSV into DataFrame
beeColonyCensusDataByCounty_file = "data/beeColonyCensusDataByCounty.csv"
beeColonyCensusDataByCounty_df = pd.read_csv(beeColonyCensusDataByCounty_file)
beeColonyCensusDataByCounty_df

,Year,Period,State,State ANSI,Ag District,Ag District Code,County,County ANSI,Value,CV (%)
0,2012,END OF DEC,ALABAMA,1,BLACK BELT,40,AUTAUGA,1.0,119,27.7
1,2012,END OF DEC,ALABAMA,1,BLACK BELT,40,DALLAS,47.0,65,27.7
2,2012,END OF DEC,ALABAMA,1,BLACK BELT,40,ELMORE,51.0,190,27.7
3,2012,END OF DEC,ALABAMA,1,BLACK BELT,40,GREENE,63.0,14,27.7
4,2012,END OF DEC,ALABAMA,1,BLACK BELT,40,HALE,65.0,10,27.7
...,...,...,...,...,...,...,...,...,...,...
7825,2002,END OF DEC,WYOMING,56,SOUTHEAST,50,GOSHEN,15.0,(D),NaN
7826,2002,END OF DEC,WYOMING,56,SOUTHEAST,50,LARAMIE,21.0,(D),NaN
7827,2002,END OF DEC,WYOMING,56,SOUTHEAST,50,PLATTE,31.0,(D),NaN
7828,2002,END OF DEC,WYOMING,56,WEST,30,LINCOLN,23.0,(D),NaN


In [3]:
# Create new data for CensusDataByCounty_df w selected columns - Removing 'Period' since all rows have the same value
CensusDataByCounty_df = beeColonyCensusDataByCounty_df[[
    'Year', 'State', 'State ANSI', 'Ag District', 'Ag District Code', 'County', 'County ANSI', 'Value', 'CV (%)']].copy()
CensusDataByCounty_rename_df = CensusDataByCounty_df.rename(columns={'CV (%)':'CV_pct'})
CensusDataByCounty_rename_df

,Year,State,State ANSI,Ag District,Ag District Code,County,County ANSI,Value,CV_pct
0,2012,ALABAMA,1,BLACK BELT,40,AUTAUGA,1.0,119,27.7
1,2012,ALABAMA,1,BLACK BELT,40,DALLAS,47.0,65,27.7
2,2012,ALABAMA,1,BLACK BELT,40,ELMORE,51.0,190,27.7
3,2012,ALABAMA,1,BLACK BELT,40,GREENE,63.0,14,27.7
4,2012,ALABAMA,1,BLACK BELT,40,HALE,65.0,10,27.7
...,...,...,...,...,...,...,...,...,...
7825,2002,WYOMING,56,SOUTHEAST,50,GOSHEN,15.0,(D),NaN
7826,2002,WYOMING,56,SOUTHEAST,50,LARAMIE,21.0,(D),NaN
7827,2002,WYOMING,56,SOUTHEAST,50,PLATTE,31.0,(D),NaN
7828,2002,WYOMING,56,WEST,30,LINCOLN,23.0,(D),NaN


In [4]:
# Drop rows with NaN values
CensusDataByCounty_rename_df = CensusDataByCounty_rename_df.dropna(how='any')

# Exclude data rows with '(D)' or '(H)' in Value & CV (%) columns
CensusDataByCounty_rename_df = CensusDataByCounty_rename_df.loc[(CensusDataByCounty_rename_df['Value'] != '(D)') & (CensusDataByCounty_rename_df['CV_pct'] != '(H)') & (CensusDataByCounty_rename_df['CV_pct'] != '(D)')]

In [5]:
# Convert columns with string of numbers to float/int
CensusDataByCounty_rename_df['Value'] = CensusDataByCounty_rename_df.Value.str.replace(',', '').astype(int)
CensusDataByCounty_rename_df['CV_pct'] = CensusDataByCounty_rename_df['CV_pct'].astype(float)

# CensusDataByCounty_df.dtypes
CensusDataByCounty_rename_df

,Year,State,State ANSI,Ag District,Ag District Code,County,County ANSI,Value,CV_pct
0,2012,ALABAMA,1,BLACK BELT,40,AUTAUGA,1.0,119,27.7
1,2012,ALABAMA,1,BLACK BELT,40,DALLAS,47.0,65,27.7
2,2012,ALABAMA,1,BLACK BELT,40,ELMORE,51.0,190,27.7
3,2012,ALABAMA,1,BLACK BELT,40,GREENE,63.0,14,27.7
4,2012,ALABAMA,1,BLACK BELT,40,HALE,65.0,10,27.7
...,...,...,...,...,...,...,...,...,...
2752,2012,WYOMING,56,SOUTH CENTRAL,40,NATRONA,25.0,8253,55.5
2753,2012,WYOMING,56,SOUTH CENTRAL,40,SWEETWATER,37.0,244,55.5
2754,2012,WYOMING,56,SOUTHEAST,50,CONVERSE,9.0,8,55.5
2756,2012,WYOMING,56,SOUTHEAST,50,LARAMIE,21.0,158,55.5


In [6]:
# Add an id column
CensusDataByCounty_rename_df.reset_index(drop=False, inplace=True)
CensusDataByCounty_rename_df = CensusDataByCounty_rename_df.rename(columns={'index':'id'})
CensusDataByCounty_rename_df.set_index('id', inplace=True)

CensusDataByCounty_rename_df.dtypes

Year                  int64
State                object
State ANSI            int64
Ag District          object
Ag District Code      int64
County               object
County ANSI         float64
Value                 int32
CV_pct              float64
dtype: object

Bees Colony Loss Data

In [7]:
# Store beeColonyLoss xlsx into DataFrame
beeColonyLoss_file = "data/beeColonyLossCSV.csv"
beeColonyLoss_df = pd.read_csv(beeColonyLoss_file)
beeColonyLoss_df

,Year,Season,State,Total Annual Loss,Beekeepers,Beekeepers Exclusive to State,Colonies,Colonies Exclusive to State
0,2016/17,Annual,Massachusetts,15.90%,87,94.30%,27186,2.30%
1,2016/17,Annual,Montana,17.10%,21,52.40%,35905,0.30%
2,2016/17,Annual,Nevada,23.00%,13,92.30%,2512,5.20%
3,2016/17,Annual,Maine,23.30%,65,93.80%,41102,1.40%
4,2016/17,Annual,Wyoming,23.40%,18,77.80%,6521,1.40%
...,...,...,...,...,...,...,...,...
360,2010/11,Annual,Puerto Rico,NaN,1,100.00%,12,100.00%
361,2010/11,Annual,Alaska,NaN,2,50.00%,6,100.00%
362,2010/11,Annual,District of Columbia,NaN,1,100.00%,4,100.00%
363,2010/11,Annual,Wyoming,NaN,2,100.00%,3883,100.00%


In [8]:
# Create new data for ColonyLoss_df w selected columns - Removing 'Season' since all rows have the same value
ColonyLoss_df = beeColonyLoss_df[['Year', 'State', 'Total Annual Loss', 'Beekeepers',
       'Beekeepers Exclusive to State', 'Colonies',
       'Colonies Exclusive to State']].copy()

ColonyLoss_renamed_df = ColonyLoss_df.rename(columns={
    'Total Annual Loss':'Total Annual Loss_pct',
    'Beekeepers Exclusive to State': 'Beekeepers Exclusive to State_pct',
    'Colonies Exclusive to State': 'Colonies Exclusive to State_pct'
})

In [9]:
# Drop rows with NaN values
ColonyLoss_renamed_df = ColonyLoss_renamed_df.dropna(how='any')
ColonyLoss_renamed_df

,Year,State,Total Annual Loss_pct,Beekeepers,Beekeepers Exclusive to State_pct,Colonies,Colonies Exclusive to State_pct
0,2016/17,Massachusetts,15.90%,87,94.30%,27186,2.30%
1,2016/17,Montana,17.10%,21,52.40%,35905,0.30%
2,2016/17,Nevada,23.00%,13,92.30%,2512,5.20%
3,2016/17,Maine,23.30%,65,93.80%,41102,1.40%
4,2016/17,Wyoming,23.40%,18,77.80%,6521,1.40%
...,...,...,...,...,...,...,...
355,2010/11,West Virginia,78.60%,25,92.00%,509,48.70%
356,2010/11,Iowa,80.70%,14,85.70%,1037,68.80%
357,2010/11,Michigan,83.50%,99,97.00%,10320,21.90%
358,2010/11,Georgia,84.70%,61,96.70%,5923,20.80%


In [10]:
# Convert columns with string of numbers to float
ColonyLoss_renamed_df['Total Annual Loss_pct'] = ColonyLoss_renamed_df['Total Annual Loss_pct'].str.replace('%', '').astype(float)
ColonyLoss_renamed_df['Beekeepers Exclusive to State_pct'] = ColonyLoss_renamed_df['Beekeepers Exclusive to State_pct'].str.replace('%', '').astype(float)
ColonyLoss_renamed_df['Colonies Exclusive to State_pct'] = ColonyLoss_renamed_df['Colonies Exclusive to State_pct'].str.replace('%', '').astype(float)

# ColonyLoss_renamed_df.dtypes
ColonyLoss_renamed_df

,Year,State,Total Annual Loss_pct,Beekeepers,Beekeepers Exclusive to State_pct,Colonies,Colonies Exclusive to State_pct
0,2016/17,Massachusetts,15.9,87,94.3,27186,2.3
1,2016/17,Montana,17.1,21,52.4,35905,0.3
2,2016/17,Nevada,23.0,13,92.3,2512,5.2
3,2016/17,Maine,23.3,65,93.8,41102,1.4
4,2016/17,Wyoming,23.4,18,77.8,6521,1.4
...,...,...,...,...,...,...,...
355,2010/11,West Virginia,78.6,25,92.0,509,48.7
356,2010/11,Iowa,80.7,14,85.7,1037,68.8
357,2010/11,Michigan,83.5,99,97.0,10320,21.9
358,2010/11,Georgia,84.7,61,96.7,5923,20.8


In [11]:
# Cleaning up the year column to show a single year
ColonyLoss_renamed_df['Year'] = ColonyLoss_renamed_df['Year'].str.slice(0, 4)
ColonyLoss_renamed_df['Year'] = ColonyLoss_renamed_df['Year'].astype(int)
ColonyLoss_renamed_df

,Year,State,Total Annual Loss_pct,Beekeepers,Beekeepers Exclusive to State_pct,Colonies,Colonies Exclusive to State_pct
0,2016,Massachusetts,15.9,87,94.3,27186,2.3
1,2016,Montana,17.1,21,52.4,35905,0.3
2,2016,Nevada,23.0,13,92.3,2512,5.2
3,2016,Maine,23.3,65,93.8,41102,1.4
4,2016,Wyoming,23.4,18,77.8,6521,1.4
...,...,...,...,...,...,...,...
355,2010,West Virginia,78.6,25,92.0,509,48.7
356,2010,Iowa,80.7,14,85.7,1037,68.8
357,2010,Michigan,83.5,99,97.0,10320,21.9
358,2010,Georgia,84.7,61,96.7,5923,20.8


In [12]:
# Add an id column
ColonyLoss_renamed_df.reset_index(drop=False, inplace=True)
ColonyLoss_renamed_df = ColonyLoss_renamed_df.rename(columns={'index':'id'})
ColonyLoss_renamed_df.set_index('id', inplace=True)

ColonyLoss_renamed_df.dtypes

Year                                   int32
State                                 object
Total Annual Loss_pct                float64
Beekeepers                             int64
Beekeepers Exclusive to State_pct    float64
Colonies                               int64
Colonies Exclusive to State_pct      float64
dtype: object

Bee Colony Survey Data By State

In [13]:
# Store beeColonySurveyDataByState csv into DataFrame
beeColonySurveyDataByState = "data/beeColonySurveyDataByState.csv"
beeColonySurveyDataByState_df = pd.read_csv(beeColonySurveyDataByState)
beeColonySurveyDataByState_df

,Year,Period,Week Ending,State,State ANSI,Watershed,Data Item,Value,CV (%)
0,2017,JAN THRU MAR,NaN,ALABAMA,1,NaN,ADDED & REPLACED,570,NaN
1,2017,JAN THRU MAR,NaN,ARIZONA,4,NaN,ADDED & REPLACED,"2,900",NaN
2,2017,JAN THRU MAR,NaN,ARKANSAS,5,NaN,ADDED & REPLACED,430,NaN
3,2017,JAN THRU MAR,NaN,CALIFORNIA,6,NaN,ADDED & REPLACED,"215,000",NaN
4,2017,JAN THRU MAR,NaN,COLORADO,8,NaN,ADDED & REPLACED,100,NaN
...,...,...,...,...,...,...,...,...,...
3391,1987,MARKETING YEAR,NaN,VIRGINIA,51,NaN,INVENTORY,"25,000",NaN
3392,1987,MARKETING YEAR,NaN,WASHINGTON,53,NaN,INVENTORY,"75,000",NaN
3393,1987,MARKETING YEAR,NaN,WEST VIRGINIA,54,NaN,INVENTORY,"21,000",NaN
3394,1987,MARKETING YEAR,NaN,WISCONSIN,55,NaN,INVENTORY,"92,000",NaN


In [14]:
# Create new data for SurveyDataByState_df w selected columns - removing 'Week Ending', 'Watershed', 'CV (%)' b/c none of the rows have data
SurveyDataByState_df = beeColonySurveyDataByState_df[[
    'Year', 'Period', 'State', 'State ANSI', 'Data Item', 'Value']].copy()

# Note: There's no NaN values
SurveyDataByState_df = SurveyDataByState_df.dropna(how='any') 
SurveyDataByState_df

,Year,Period,State,State ANSI,Data Item,Value
0,2017,JAN THRU MAR,ALABAMA,1,ADDED & REPLACED,570
1,2017,JAN THRU MAR,ARIZONA,4,ADDED & REPLACED,"2,900"
2,2017,JAN THRU MAR,ARKANSAS,5,ADDED & REPLACED,430
3,2017,JAN THRU MAR,CALIFORNIA,6,ADDED & REPLACED,"215,000"
4,2017,JAN THRU MAR,COLORADO,8,ADDED & REPLACED,100
...,...,...,...,...,...,...
3391,1987,MARKETING YEAR,VIRGINIA,51,INVENTORY,"25,000"
3392,1987,MARKETING YEAR,WASHINGTON,53,INVENTORY,"75,000"
3393,1987,MARKETING YEAR,WEST VIRGINIA,54,INVENTORY,"21,000"
3394,1987,MARKETING YEAR,WISCONSIN,55,INVENTORY,"92,000"


In [15]:
# Convert columns with string of numbers to float
SurveyDataByState_df['Value'] = SurveyDataByState_df.Value.str.replace(',', '').astype(int)

SurveyDataByState_df

,Year,Period,State,State ANSI,Data Item,Value
0,2017,JAN THRU MAR,ALABAMA,1,ADDED & REPLACED,570
1,2017,JAN THRU MAR,ARIZONA,4,ADDED & REPLACED,2900
2,2017,JAN THRU MAR,ARKANSAS,5,ADDED & REPLACED,430
3,2017,JAN THRU MAR,CALIFORNIA,6,ADDED & REPLACED,215000
4,2017,JAN THRU MAR,COLORADO,8,ADDED & REPLACED,100
...,...,...,...,...,...,...
3391,1987,MARKETING YEAR,VIRGINIA,51,INVENTORY,25000
3392,1987,MARKETING YEAR,WASHINGTON,53,INVENTORY,75000
3393,1987,MARKETING YEAR,WEST VIRGINIA,54,INVENTORY,21000
3394,1987,MARKETING YEAR,WISCONSIN,55,INVENTORY,92000


In [16]:
# Add an id column
SurveyDataByState_df.reset_index(drop=False, inplace=True)
SurveyDataByState_df = SurveyDataByState_df.rename(columns={'index':'id'})
SurveyDataByState_df.set_index('id', inplace=True)

SurveyDataByState_df.dtypes

Year           int64
Period        object
State         object
State ANSI     int64
Data Item     object
Value          int32
dtype: object

# LOAD

In [17]:
# Create database connection
engine = create_engine("sqlite:///data/bee_colony.sqlite")
conn = engine.connect()

In [25]:
# Confirm tables
engine.table_names()

['census_county', 'census_state', 'colonyloss']

In [19]:
# Load SurveyDataByState_df into database
SurveyDataByState_df.to_sql(name='census_state', con=engine, if_exists='append', index=True)

In [20]:
# Load CensusDataByCounty_df into database
CensusDataByCounty_rename_df.to_sql(name='census_county', con=engine, if_exists='append', index=True)

In [21]:
# Load ColonyLoss_renamed_df into database
ColonyLoss_renamed_df.to_sql(name='colonyloss', con=engine, if_exists='append', index=True)

In [22]:
# Verify we can query from the database
pd.read_sql_query('select * from census_state', con=engine).head()

,id,Year,Period,State,State ANSI,Data Item,Value
0,0,2017,JAN THRU MAR,ALABAMA,1,ADDED & REPLACED,570
1,1,2017,JAN THRU MAR,ARIZONA,4,ADDED & REPLACED,2900
2,2,2017,JAN THRU MAR,ARKANSAS,5,ADDED & REPLACED,430
3,3,2017,JAN THRU MAR,CALIFORNIA,6,ADDED & REPLACED,215000
4,4,2017,JAN THRU MAR,COLORADO,8,ADDED & REPLACED,100


In [23]:
pd.read_sql_query('select * from census_county', con=engine).head()

,id,Year,State,State ANSI,Ag District,Ag District Code,County,County ANSI,Value,CV_pct
0,0,2012,ALABAMA,1,BLACK BELT,40,AUTAUGA,1.0,119,27.7
1,1,2012,ALABAMA,1,BLACK BELT,40,DALLAS,47.0,65,27.7
2,2,2012,ALABAMA,1,BLACK BELT,40,ELMORE,51.0,190,27.7
3,3,2012,ALABAMA,1,BLACK BELT,40,GREENE,63.0,14,27.7
4,4,2012,ALABAMA,1,BLACK BELT,40,HALE,65.0,10,27.7


In [24]:
pd.read_sql_query('select * from colonyloss', con=engine).head()

,id,Year,State,Total Annual Loss_pct,Beekeepers,Beekeepers Exclusive to State_pct,Colonies,Colonies Exclusive to State_pct
0,0,2016,Massachusetts,15.9,87,94.3,27186,2.3
1,1,2016,Montana,17.1,21,52.4,35905,0.3
2,2,2016,Nevada,23.0,13,92.3,2512,5.2
3,3,2016,Maine,23.3,65,93.8,41102,1.4
4,4,2016,Wyoming,23.4,18,77.8,6521,1.4
